# ForceSMIP Example: Load Data Only

**Notebook author:** Robb Jnglin Wills (r.jnglinwills@usys.ethz.ch)

The goal of this notebook is to provide a simple notebook for loading in the ForceSMIP training and evaluation data and taking anomalies from the full-period climatology. This is meant to be a starting point for the development of other notebooks. 

#### Outline:

* Notes on setup with conda
* Import of key packages
* User-specified options
* Loading in the ForceSMIP data
    * Defining a function to read in data
    * Loading monthly anomaly maps for the evaluation data
    * Loading monthly anomaly maps for the training data
    * Calculating the ensemble-mean monthly anomaly maps for the training data

### Data structure, training and evaluation data

This notebook loads the ForceSMIP **training data** and the ForceSMIP **evaluation data**. For the training data, we have many ensemble members, and we can compute the forced response as the ensemble average. For the evaluation data, we only have one realization from each model, just as we only have one realization of the real world. The challenge of ForceSMIP is to come up with methods that can approximate the forced response from these single realizations, removing internal variability as we would if we were able to take an ensemble mean.

## Some setup notes

This setup assumes that you have anaconda installed. If you do not, you can install miniconda (from [here](https://docs.conda.io/en/main/miniconda.html)).

To create this environment we ran `conda create -n forcesmip -c conda-forge xcdat xesmf scikit-learn scipy eofs matplotlib cartopy nc-time-axis ipython ipykernel tensorflow python=3.9 -y`

> Note: You may need to check your shell to activate conda. To determine your shell, you can run `echo $SHELL`. You can then activate conda (based on your shell type, e.g., `conda init bash`). You may subsequently need to re-source your shell (e.g., `source ~/.bashrc` for bash).

Activate your environment with: `conda activate forcesmip`

> Note: Alternatively, you can try `source activate forcesmip`

If you'd like to be able to use this environment with Jupyter, you need to install it with:

`python -m ipykernel install --user --name forcesmip --display-name forcesmip`

### Import packages (code starts here)

In [1]:
# I/O / data wrangling
import os
import glob
import re
import numpy as np
import xarray as xr
import xcdat as xc

# runtime metrics
import time as clocktime

# plotting
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point

# define a lambda function to perform natural sort
natsort = lambda s: [int(t) if t.isdigit() else t.lower() for t in re.split("(\d+)", s)]

### Define some static mappings for CMIP/ForceSMIP data

This is just some helper information to helps us search for data and reshape it. All ForceSMIP data is on a 2.5 x 2.5 degree lat/lon grid. 

In [2]:
cmipTable = {
    "pr": "Amon",
    "psl": "Amon",
    "tas": "Amon",
    "zmta": "Amon",
    "tos": "Omon",
    "siconc": "OImon",
    "monmaxpr": "Aday",
    "monmaxtasmax": "Aday",
    "monmintasmin": "Aday",
}
cmipVar = {
    "pr": "pr",
    "psl": "psl",
    "tas": "tas",
    "zmta": "ta",
    "tos": "tos",
    "siconc": "siconc",
    "monmaxpr": "pr",
    "monmaxtasmax": "tasmax",
    "monmintasmin": "tasmin",
}
evalPeriods = {
    "Tier1": ("1950-01-01", "2022-12-31"),
    "Tier2": ("1900-01-01", "2022-12-31"),
    "Tier3": ("1979-01-01", "2022-12-31"),
}
nlat = 72
nlon = 144

### Define user-specified parameters

In [3]:
root_dir = "/net/krypton/climdyn_nobackup/FTP/ForceSMIP/"  # path to forcesmip data (ETH)
#root_dir = "/glade/campaign/cgd/cas/asphilli/ForceSMIP/"  # path to forcesmip data (NCAR)

outdir = "ForceSMIP_output/"  # directory where output data should be saved

ncvar = "tas"  # CMIP variable name to be used

# choose models for training
# choices include: 'CESM2', 'CanESM5', 'MIROC-ES2L', 'MIROC6', 'MPI-ESM1-2-LR'
training_models = ["CESM2","CanESM5","MIROC-ES2L","MIROC6","MPI-ESM1-2-LR"]
n_members = 10  # number of members for training

# choose evaluation data
eval_tier = "Tier1"  # Tier1, Tier2, or Tier3

# no need to modify the training or reference period for LFCA
tv_time_period = evalPeriods[eval_tier] # ("1950-01-01","2022-12-30")  # period of time to consider data for training
reference_period = tv_time_period # anomalies will be with respect to mean over entire period (convectional for LFCA)

### Define a function to read in data

We're going to loop over many models and realizations for training and evaluation data. To make this more readable and to reduce repeating code, we are going to define a function to do this operation.

In [4]:
def load_realization(fn, vid, time_period, reference_period):
    """
    load_realization(fn, vid, time_period, reference_period)
    
    Function loads in data for a given file, fn, and variable, vid. It
    selects data for a given time_period and calculates the anomalies
    relative to a user-defined reference_period. The function returns arrays
    of the dimensions (time, lat, lon), the 3D anomaly map, and the global
    mean time series. 
    
    Inputs:
    -------
    fn (str) : filename
    vid (str) : variable id
    time_period (tuple(str, str)) : tuple of the start and end of the time period
                                    e.g., ("1900-01-01", "1949-12-31")
    reference_period (tuple(str, str)) : tuple of the start and end of the reference period
                                         used to calculate anomalies e.g., ("1900-01-01", "1949-12-31")
                                         
    Returns:
    --------
    ts_3d (xr.DataArray) : monthly average anomaly values [time, lat, lon]
    ts_gm (xr.DataArray) : monthly average, global mean anomaly values
    """
    # open dataset
    ds = xc.open_dataset(fn)
    # if specified, subset training/validation data to specific period
    if tv_time_period is not None:
        ds = ds.sel(time=slice(time_period[0], time_period[1]))
    # get departures
    ds = ds.temporal.departures(vid, freq="month", reference_period=reference_period)
    # If you wanted annual averages instead, you could use the following:
    # ds = ds.temporal.group_average(vid, freq="year", weighted=False)
    ts_3d = ds[vid]
    # take spatial average
    ds = ds.spatial.average(vid)
    ts_gm = ds[vid]
    # clean up 
    ds.close()
    # return values
    return ts_3d, ts_gm

### Read in evaluation data

In [5]:
# first we search for the evaluation data
epath = "/".join([root_dir, "Evaluation-" + eval_tier, cmipTable[ncvar], ncvar])
efiles = glob.glob(epath + "/*.nc")
efiles = sorted(efiles, key=natsort)

# initialize dictionary to store data
evaluation_anomaly_maps = {}
missing_data_mask = {}
vid = cmipVar[ncvar]

# loop over evaluation files
for im, fn in enumerate(efiles):
    # get evaluation identifier
    model = fn.split("/")[-1].split("_")[2].split(".")[0]
    # print progress
    print(str(im + 1) + " / " + str(len(efiles)) + ": " + model)
    # read in data for realization
    ts_3d, ts_gm = load_realization(fn, vid, tv_time_period, reference_period)
    # store anomaly map
    evaluation_anomaly_maps[model] = ts_3d
    
    # create mask for missing data
    tmp = np.mean(ts_3d, axis=0)
    missing_data_mask[model] = np.where(np.isnan(tmp), np.nan, 1)

1 / 10: 1A
2 / 10: 1B
3 / 10: 1C
4 / 10: 1D
5 / 10: 1E
6 / 10: 1F
7 / 10: 1G
8 / 10: 1H
9 / 10: 1I
10 / 10: 1J


### Loop over training models and retrieve monthly anomaly maps

The script stores all of the time series but not all of the full `[time, lat, lon]` timeseries (the number of full fields is specified by the `n_members` parameter above).

In [6]:
# initialize dictionary to store data
global_mean_timeseries = {}
anomaly_maps = {}
anomaly_map_emean = {}
missing_data_mask_training = {}
vid = cmipVar[ncvar]
# loop over models
models = training_models
for im, model in enumerate(models):
    # start timer
    stime = clocktime.time()
    # initialize nested dictionary for model data
    global_mean_timeseries[model] = {}
    anomaly_maps[model] = {}
    anomaly_map_emean[model] = 0
    missing_data_mask_training[model] = {}
    # get model files
    mpath = "/".join([root_dir, "Training", cmipTable[ncvar], ncvar, model])
    mfiles = glob.glob(mpath + "/*.nc")
    # parse file names to get list of model members
    # CESM2 has a non-CMIP naming convention
    if model == "CESM2":
        members = [p.split("ssp370_")[-1].split(".1880")[0] for p in mfiles]
    else:
        members = [p.split("_")[-1].split(".")[0] for p in mfiles]
    members = sorted(members, key=natsort)
    # print progress
    print(str(im + 1) + " / " + str(len(models)) + ": " + model + " (" + str(len(members)) + " members)")
    # loop over model members
    for imm, member in enumerate(members):
        # define member filename
        fn = glob.glob(mpath + "/*_" + member + ".*.nc")
        # make sure filename is unique
        if len(fn) != 1:
            raise ValueError("Unexpected number of model members")
        else:
            fn = fn[0]
        # load data for realization
        ts_3d, ts_gm = load_realization(fn, vid, tv_time_period, reference_period)
        # store first N anomaly maps from training models
        # also store all data for the validation models
        if ((imm < n_members) & (model in training_models)):# | (model in validation_models):
            # store masked data in array
            anomaly_maps[model][member] = ts_3d
        # ensemble mean anomaly map
        anomaly_map_emean[model] = anomaly_map_emean[model] + ts_3d/len(members)
        # save global mean time series for all model members
        global_mean_timeseries[model][member] = ts_gm
        # create mask for missing data
        tmp = np.mean(ts_3d, axis=0)
        missing_data_mask_training[model] = np.where(np.isnan(tmp), np.nan, 1)
        # update progress
        print(".", end="")
    # print time elapse for model
    etime = clocktime.time()
    print()
    print("Time elapsed: " + str(etime - stime) + " seconds")
    print()

1 / 5: CESM2 (50 members)
..................................................
Time elapsed: 54.85980224609375 seconds

2 / 5: CanESM5 (25 members)
.........................
Time elapsed: 36.48008680343628 seconds

3 / 5: MIROC-ES2L (30 members)
..............................
Time elapsed: 47.33926296234131 seconds

4 / 5: MIROC6 (50 members)
..................................................
Time elapsed: 72.03572511672974 seconds

5 / 5: MPI-ESM1-2-LR (30 members)
..............................
Time elapsed: 40.693753719329834 seconds



## Now the training and evaluation data are loaded as data arrays in dictionaries

Explore the data and try your own method from here.

In [19]:
# training models
training_models

['CESM2', 'CanESM5', 'MIROC-ES2L', 'MIROC6', 'MPI-ESM1-2-LR']

In [20]:
# example of how to get the ensemble member names for CESM2, one of the training models
k = list(anomaly_maps['CESM2'].keys())

In [21]:
# example of how to extract the anomaly map for the first training map of CESM2
anomalies = anomaly_maps['CESM2'][k[0]]
# the ensemble mean anomalies have the same size, but are the mean over all members of the chosen training model
anomalies_emean = anomaly_map_emean['CESM2']

In [22]:
# display anomalies
anomalies

<xarray.DataArray 'tas' (time: 876, lat: 72, lon: 144)>
array([[[-2.33780785, -2.34212734, -2.33305004, ..., -2.31544265,
         -2.31493785, -2.32485042],
        [-1.74593406, -1.71768565, -1.69741257, ..., -1.75956831,
         -1.74716918, -1.74227571],
        [-1.46507848, -1.44747423, -1.39469889, ..., -1.30001538,
         -1.37599537, -1.46371021],
        ...,
        [ 1.13785961,  1.13670286,  1.1415654 , ...,  1.135863  ,
          1.16550728,  1.15063832],
        [ 1.77224982,  1.69760675,  1.61240272, ...,  2.03593319,
          1.95397009,  1.85876548],
        [ 1.25352854,  1.24353278,  1.22203189, ...,  1.31894987,
          1.30010025,  1.27212274]],

       [[-1.43442943, -1.423121  , -1.39880998, ..., -1.52851721,
         -1.50156204, -1.45912212],
        [-2.33598014, -2.2085797 , -2.08087932, ..., -2.58787537,
         -2.52826847, -2.41728712],
        [-2.31532183, -2.39728536, -2.50095211, ..., -1.8674214 ,
         -1.98827414, -2.17365087],
...
        [ 3.48680407,  3.72401595,  4.0695922 , ...,  1.4532339 ,
          1.99425872,  2.97643249],
        [ 3.52698715,  3.7025274 ,  3.92096282, ...,  3.03566711,
          3.15720587,  3.34568034],
        [ 2.93620635,  2.94565823,  2.96349784, ...,  2.85878158,
          2.88561426,  2.91909393]],

       [[-2.36121483, -2.36407513, -2.37000201, ..., -2.31170445,
         -2.32581131, -2.35068292],
        [-1.68219705, -1.72328897, -1.84809081, ..., -1.68282454,
         -1.7072246 , -1.70924607],
        [-0.83990897, -0.87921937, -0.97136636, ..., -0.76732416,
         -0.72567645, -0.76315412],
        ...,
        [-3.35298742, -3.5338233 , -3.81477084, ..., -2.39969499,
         -2.53896656, -3.0276021 ],
        [-2.37179691, -2.38773022, -2.3538787 , ..., -2.18341984,
         -2.26146917, -2.32929449],
        [-0.69813788, -0.69928376, -0.70335493, ..., -0.70144507,
         -0.69754321, -0.69651376]]])
Coordinates:
  * time     (time) object 1950-01-16 12:00:00 ... 2022-12-16 12:00:00
  * lat      (lat) float64 -88.75 -86.25 -83.75 -81.25 ... 83.75 86.25 88.75
  * lon      (lon) float64 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
Attributes:
    long_name:     Reference height temperature
    units:         K
    cell_methods:  time: mean
    operation:     temporal_avg
    mode:          departures
    freq:          month
    weighted:      True

In [23]:
# extract anomalies maps from evaluation member 1A
# note that tier1 evaluation members are named 1A, 1B, ... 1J, and similar for tier 2 and tier 3
evaluation_anomalies = evaluation_anomaly_maps['1A']

In [24]:
# display evaluation anomalies
evaluation_anomalies

<xarray.DataArray 'tas' (time: 876, lat: 72, lon: 144)>
array([[[ 2.22007962,  2.16801841,  2.18887904, ...,  2.21339921,
          2.24446786,  2.26036235],
        [ 2.79571096,  2.77549374,  2.76214183, ...,  2.76079443,
          2.8510082 ,  2.81524101],
        [ 2.26409496,  2.18215187,  2.10406203, ...,  2.49993574,
          2.42652726,  2.33622071],
        ...,
        [-7.0224205 , -7.23603685, -7.36803484, ..., -6.10702977,
         -6.45272862, -6.76909605],
        [-4.56895688, -4.6429312 , -4.75812274, ..., -4.48399656,
         -4.61033153, -4.6633843 ],
        [-2.41253861, -2.39676555, -2.37985434, ..., -2.47744356,
         -2.47116728, -2.45100992]],

       [[ 0.93749613,  0.88032624,  0.93222948, ...,  1.02245846,
          0.99879589,  0.98169064],
        [ 1.45381535,  1.31962415,  1.25301937, ...,  1.4543875 ,
          1.50932388,  1.48991203],
        [ 2.29474676,  2.28756393,  2.22605676, ...,  2.02619866,
          2.14167835,  2.21824135],
...
        [-2.7827965 , -2.66927603, -2.49980953, ..., -2.91251649,
         -2.91124306, -2.87273756],
        [-0.91584671, -0.80642469, -0.67359547, ..., -1.163381  ,
         -1.07829428, -1.02262005],
        [ 1.35370195,  1.3781469 ,  1.40556517, ...,  1.31387891,
          1.33169362,  1.33744102]],

       [[-1.80688922, -1.76666199, -1.71510911, ..., -1.890938  ,
         -1.83467338, -1.80803475],
        [-2.38328243, -2.24900122, -2.15147902, ..., -2.69572454,
         -2.57511935, -2.47447839],
        [-1.73124902, -1.65932998, -1.58802879, ..., -2.08066062,
         -1.91432143, -1.80544713],
        ...,
        [ 9.68594609,  9.31857019,  8.89418763, ..., 10.36729583,
         10.20970026,  9.98452008],
        [ 9.73104666,  9.64881584,  9.54263706, ...,  9.82037808,
          9.8393142 ,  9.79265549],
        [ 9.24446793,  9.22630737,  9.20215929, ...,  9.2907559 ,
          9.28566131,  9.25710949]]])
Coordinates:
  * time     (time) object 1950-01-16 12:00:00 ... 2022-12-16 12:00:00
  * lat      (lat) float64 -88.75 -86.25 -83.75 -81.25 ... 83.75 86.25 88.75
  * lon      (lon) float64 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
Attributes:
    long_name:  Near-Surface Air Temperature
    units:      K
    operation:  temporal_avg
    mode:       departures
    freq:       month
    weighted:   True